In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

In [2]:
%load_ext autoreload
%autoreload 1
%aimport analyze

In [ ]:
summit = analyze.Analysis('results/summit')

In [6]:
print(summit)

Analysis for Celeritas v0.1.2+fb3f5506 on summit


In [7]:
summed = analyze.summarize_instances(summit.result[[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time']])

In [9]:
occupancy = summit.results['num_steps'] / (summit.results['num_step_iters'] * summit.input['max_num_tracks'])

Index(['action_times', 'active_hwm', 'avg_steps_per_primary',
       'avg_time_per_primary', 'avg_time_per_step', 'emptying_step',
       'num_primaries', 'num_step_iters', 'num_steps', 'pre_emptying_time',
       'queue_hwm', 'total_time', 'unconverged', 'exception', 'failure'],
      dtype='object')

In [8]:
summed

avg_steps_per_primary                 \
summary                                           count           mean   
problem              geo     arch                                        
cms2018              vecgeom cpu                    6.0   10088.901164   
                             gpu                    6.0    4114.109383   
cms2018+field+msc    vecgeom cpu                    6.0  524288.000000   
                             gpu                    6.0   10233.794083   
simple-cms+field     orange  cpu                    0.0            NaN   
                             gpu                    0.0            NaN   
simple-cms+field+msc orange  cpu                    0.0            NaN   
                             gpu                    0.0            NaN   
                     vecgeom cpu                    6.0    6969.623169   
                             gpu                    6.0    2826.762183   
simple-cms+msc       orange  cpu                    6.0    6879.932373   
                             gpu                    6.0    2818.600433   
testem15             orange  cpu                    6.0      46.485392   
                             gpu                    6.0      19.037267   
testem15+field       orange  cpu                    6.0      46.306519   
                             gpu                    6.0      19.015150   
testem15+field+msc   orange  cpu                    6.0      46.455973   
                             gpu                    6.0      19.016683   
                     vecgeom cpu                    6.0      46.455973   
                             gpu                    6.0      19.016683   
testem3-flat         orange  cpu                    6.0  112504.021037   
                             gpu                    0.0            NaN   
                     vecgeom cpu                    6.0  112504.021037   
                             gpu                    0.0            NaN   
testem3-flat+field   orange  cpu                    2.0  112546.124390   
                             gpu                    0.0            NaN   
testem3-flat+msc     orange  cpu                    4.0  149644.703796   
                             gpu                    0.0            NaN   

                                             avg_time_per_primary            \
summary                                  std                count      mean   
problem              geo     arch                                             
cms2018              vecgeom cpu   28.569940                  6.0  0.003469   
                             gpu   20.988082                  6.0  0.000319   
cms2018+field+msc    vecgeom cpu    0.000000                  6.0  0.379878   
                             gpu   61.549550                  6.0  0.002426   
simple-cms+field     orange  cpu         NaN                  0.0       NaN   
                             gpu         NaN                  0.0       NaN   
simple-cms+field+msc orange  cpu         NaN                  0.0       NaN   
                             gpu         NaN                  0.0       NaN   
                     vecgeom cpu   87.213186                  6.0  0.005912   
                             gpu    2.516712                  6.0  0.000524   
simple-cms+msc       orange  cpu    4.866362                  6.0  0.002188   
                             gpu    1.436237                  6.0  0.000055   
testem15             orange  cpu    0.218272                  6.0  0.000019   
                             gpu    0.150565                  6.0  0.000018   
testem15+field       orange  cpu    0.315915                  6.0  0.000019   
                             gpu    0.067161                  6.0  0.000015   
testem15+field+msc   orange  cpu    0.184653                  6.0  0.000025   
                             gpu    0.111390                  6.0  0.000015   
                     vecgeom cpu    0.184653                  6.0  0.000025   
 

In [ ]:
def get_cpugpu_ratio(summary):
    mean = summary['mean'].unstack()
    rel  = (summary['std'] / summary['mean']).unstack()
    ratio = mean['cpu'] / mean['gpu']
    rel_err = np.sqrt(rel['cpu']**2 + rel['gpu']**2)
    return pd.DataFrame({'ratio': ratio, 're': rel_err})

In [ ]:
get_cpugpu_ratio(summed['avg_time_per_step'])

In [11]:
summit.results.index

MultiIndex([('cms2018+field+msc', 'vecgeom', 'cpu', 0),
            ('cms2018+field+msc', 'vecgeom', 'cpu', 1),
            ('cms2018+field+msc', 'vecgeom', 'cpu', 2),
            ('cms2018+field+msc', 'vecgeom', 'cpu', 3),
            ('cms2018+field+msc', 'vecgeom', 'cpu', 4),
            ('cms2018+field+msc', 'vecgeom', 'cpu', 5),
            ('cms2018+field+msc', 'vecgeom', 'gpu', 0),
            ('cms2018+field+msc', 'vecgeom', 'gpu', 1),
            ('cms2018+field+msc', 'vecgeom', 'gpu', 2),
            ('cms2018+field+msc', 'vecgeom', 'gpu', 3),
            ...
            (     'testem3-flat', 'vecgeom', 'cpu', 2),
            (     'testem3-flat', 'vecgeom', 'cpu', 3),
            (     'testem3-flat', 'vecgeom', 'cpu', 4),
            (     'testem3-flat', 'vecgeom', 'cpu', 5),
            (     'testem3-flat', 'vecgeom', 'gpu', 0),
            (     'testem3-flat', 'vecgeom', 'gpu', 1),
            (     'testem3-flat', 'vecgeom', 'gpu', 2),
            (     'testem3-flat'

In [ ]:
summed[['num_steps', 'unconverged']].xs('mean', axis=1, level='summary')

In [ ]:
def plot_counts(ax, out):
    blue = (.1, .1, .9)
    red = (.7, .1, .1)

    lines = []
    def plot(ax, *args, **kwargs):
        line, = ax.plot(*args, **kwargs)
        lines.append(line)

    plot(ax, out['result']['active'], '-', color=(blue + (0.5,)), label='Active')
    plot(ax, out['result']['alive'], '-', color=blue, label='Alive')
    ax.set_xlabel('Step iteration')
    ax.set_ylabel('Tracks', color=blue)

    oax = ax.twinx()
    inits = np.array(out['result']['initializers'])
    plot(oax, inits, '--', color=red, label='Queued')
    oax.axhline(out['input']['initializer_capacity'], linestyle='--', color=(red + (0.25,)))
    oax.set_ylabel('Initializers', color=red)

    max_init_idx = np.argmax(inits)
    max_init_val = inits[max_init_idx]
    text = re.sub(r'([-+.0-9]+)e\+?(-)?0*([0-9]+)', r'$\1\\times 10^{\2\3}$',
                  f'{max_init_val:.2g}')
    oax.annotate(text + ' queued', xy=(max_init_idx, max_init_val), xycoords='data',
                 xytext=(30, 10), textcoords='offset points',
                 size='x-small',
                 bbox=dict(boxstyle="round,pad=.2", fc=(0.9, 0.9, 0.9, 0.8) , ec=(0.2,)*3),
                 arrowprops=dict(arrowstyle="->", ec=red, lw=1,
                                 connectionstyle="arc3,rad=0.2"))

    oax.spines['left'].set_color(blue)
    oax.spines['right'].set_color(red)

    ax.legend(lines, [l.get_label() for l in lines])

    return {
        'ax': ax,
        'oax': oax,
    }

In [ ]:
with open('results/summit/cms2018+field+msc-vecgeom-gpu/0.json') as f:
    full = json.load(f)

In [ ]:
(fig, ax) = plt.subplots()
plot_counts(ax, full);